# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
# pip install docarray

  Obtaining dependency information for docarray from https://files.pythonhosted.org/packages/18/89/f9196ad5c3585cff64e81fc7d741984610b1e60ad555d46be0a11a36d0bb/docarray-0.37.0-py3-none-any.whl.metadata
  Obtaining dependency information for orjson>=3.8.2 from https://files.pythonhosted.org/packages/fb/ac/e9c196e223b939d7a9ac6f9f4d3edab78152cbc206927318fb89c3f7ff59/orjson-3.9.3-cp310-cp310-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
  Obtaining dependency information for rich>=13.1.0 from https://files.pythonhosted.org/packages/8d/5f/21a93b2ec205f4b79853ff6e838e3c99064d5dbe85ec6b05967506f14af0/rich-13.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests>=2.28.11.6 from https://files.pythonhosted.org/packages/06/9b/04bb62f11a6824df5d4568439cf0715118c265d0ffbebeb7cf4b8c9caa15/types_requests-2.31.0.2-py3-none-any.whl.metadata
  Obtaining dependency i

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [42]:
query ="Please list all jacket \
in a table in markdown and summarize each one."

In [43]:
response = index.query(query)

In [44]:
display(Markdown(response))



| Jacket | Description |
| --- | --- |
| Storm Shield Field Jacket | Washed-down utility jacket designed for effortless style and comfort indoors or out. Slightly Fitted, Softly shapes the body, Falls at low hip. 98% cotton with 2% spandex for superior stretch. Garment dyed for broken-in comfort right from the start. |
| Mountainwood Plaid Jacket | Easygoing blazer crafted in fine Italian tweed yarns. Slightly Fitted, Softly shapes the body, Falls at hip. 47% wool/33% nylon/19% silky/1% spandex, Fully lined, Dry clean. Softly textured and warm without being too heavy, Fold collar over for a classic look, or snap it up to the stand collar for a more modern look, Front flap pockets with snap closures also have zip pockets, Slash chest pockets, Faux suede elbow patches, Back vent, Imported. |
| Mountain Maven Reversible Jacket | Double down on cool, damp weather with our boys' reversible jacket. Water-resistant ripstop nylon shell reverses to 100% polyester fleece. Plush fleece reverses to rugged nylon for two options in one.

In [45]:
loader = CSVLoader(file_path=file)

In [46]:
docs = loader.load()

In [47]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [48]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [49]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [50]:
print(len(embed))

1536


In [51]:
print(embed[:5])

[-0.021913960932078386, 0.0067742067558426095, -0.018190348816400984, -0.0391482493681045, -0.014089343366938921]


In [52]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [53]:
query = "Please suggest a shirt with sunblocking"

In [54]:
docs = db.similarity_search(query)

In [55]:
len(docs)

4

In [56]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [57]:
retriever = db.as_retriever()

In [58]:
llm = ChatOpenAI(temperature = 0.0)


In [59]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [61]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


KeyboardInterrupt: 

In [62]:
display(Markdown(response))



| Jacket | Description |
| --- | --- |
| Storm Shield Field Jacket | Washed-down utility jacket designed for effortless style and comfort indoors or out. Slightly Fitted, Softly shapes the body, Falls at low hip. 98% cotton with 2% spandex for superior stretch. Garment dyed for broken-in comfort right from the start. |
| Mountainwood Plaid Jacket | Easygoing blazer crafted in fine Italian tweed yarns. Slightly Fitted, Softly shapes the body, Falls at hip. 47% wool/33% nylon/19% silky/1% spandex, Fully lined, Dry clean. Softly textured and warm without being too heavy, Fold collar over for a classic look, or snap it up to the stand collar for a more modern look, Front flap pockets with snap closures also have zip pockets, Slash chest pockets, Faux suede elbow patches, Back vent, Imported. |
| Mountain Maven Reversible Jacket | Double down on cool, damp weather with our boys' reversible jacket. Water-resistant ripstop nylon shell reverses to 100% polyester fleece. Plush fleece reverses to rugged nylon for two options in one.

In [63]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [64]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [66]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...


KeyboardInterrupt: 

In [67]:
display(Markdown(response))



| Jacket | Description |
| --- | --- |
| Storm Shield Field Jacket | Washed-down utility jacket designed for effortless style and comfort indoors or out. Slightly Fitted, Softly shapes the body, Falls at low hip. 98% cotton with 2% spandex for superior stretch. Garment dyed for broken-in comfort right from the start. |
| Mountainwood Plaid Jacket | Easygoing blazer crafted in fine Italian tweed yarns. Slightly Fitted, Softly shapes the body, Falls at hip. 47% wool/33% nylon/19% silky/1% spandex, Fully lined, Dry clean. Softly textured and warm without being too heavy, Fold collar over for a classic look, or snap it up to the stand collar for a more modern look, Front flap pockets with snap closures also have zip pockets, Slash chest pockets, Faux suede elbow patches, Back vent, Imported. |
| Mountain Maven Reversible Jacket | Double down on cool, damp weather with our boys' reversible jacket. Water-resistant ripstop nylon shell reverses to 100% polyester fleece. Plush fleece reverses to rugged nylon for two options in one.

In [68]:
response = index.query(query, llm=llm)

KeyboardInterrupt: 

In [69]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])